# Amazon Nova

In [15]:
import json

import boto3

session = boto3.Session(profile_name="gen-ai-ops", region_name="us-east-1")
bedrock_client = session.client(service_name="bedrock")
bedrock_runtime_client = session.client(service_name="bedrock-runtime")

In [16]:
PRO_MODEL_ID = "amazon.nova-pro-v1:0"  # TEXT & VISION
LITE_MODEL_ID = "amazon.nova-lite-v1:0"  # TEXT & VISION
MICRO_MODEL_ID = "amazon.nova-micro-v1:0"  # TEXT

#### Synchronous API Call

In [17]:
system_list = [{"text": "You should respond all the messages in French."}]
message_llist = [
    {
        "role": "user",
        "content": [
            {
                "text": "Youtube is a great platform to learn coding and technology in the field of computer science."
            }
        ],
    }
]

config = {"max_new_tokens": 300, "top_p": 0.9, "top_k": 20}
request = {"messages": message_llist, "system": system_list, "inferenceConfig": config}

response = bedrock_runtime_client.invoke_model(
    modelId=LITE_MODEL_ID, body=json.dumps(request)
)
print(response)

request_id = response["ResponseMetadata"]["RequestId"]
print(request_id)

body = json.loads(response["body"].read())
print(body)

content_text = body["output"]["message"]["content"][0]["text"]
print("\n", content_text)

{'ResponseMetadata': {'RequestId': '2939cda5-7430-43e2-923e-112b3ca886d4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 01 Oct 2025 16:49:55 GMT', 'content-type': 'application/json', 'content-length': '888', 'connection': 'keep-alive', 'x-amzn-requestid': '2939cda5-7430-43e2-923e-112b3ca886d4', 'x-amzn-bedrock-invocation-latency': '1024', 'x-amzn-bedrock-cache-write-input-token-count': '0', 'x-amzn-bedrock-cache-read-input-token-count': '0', 'x-amzn-bedrock-output-token-count': '149', 'x-amzn-bedrock-input-token-count': '26'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x000001BF352F21A0>}
2939cda5-7430-43e2-923e-112b3ca886d4
{'output': {'message': {'content': [{'text': "YouTube est une excellente plateforme pour apprendre le codage et la technologie dans le domaine de l'informatique. De nombreux créateurs de contenu proposent des tutoriels détaillés, des cours complets et des explications sur divers sujets tels qu

#### Streaming API Call

In [18]:
from datetime import datetime as dt

system_list = [
    {
        "text": "Act as a writing assistant. When the user provides you with a topic, write a short story about that topic"
    }
]

message_llist = [{"role": "user", "content": [{"text": "A trip to Ladakh."}]}]

config = {"max_new_tokens": 1000, "top_p": 0.9, "top_k": 20}

request = {"messages": message_llist, "system": system_list, "inferenceConfig": config}

start_time = dt.now()

response = bedrock_runtime_client.invoke_model_with_response_stream(
    modelId=LITE_MODEL_ID, body=json.dumps(request)
)

chunk_count = 0
time_to_first_token = None

stream = response.get("body")
if stream:
    for event in stream:
        chunk = event.get("chunk")
        if chunk:
            chunk_json = json.loads(chunk.get("bytes").decode())
            content_block_delta = chunk_json.get("contentBlockDelta")
            if content_block_delta:
                if time_to_first_token is None:
                    time_to_first_token = dt.now() - start_time
                    print(f"Time to first token: {time_to_first_token}")
                chunk_count += 1
                current_time = dt.now().strftime("%d-%m-%Y %H:%M:%S:%f")
                print(content_block_delta.get("delta").get("text"), end="")
    print("\nTotal token count: ", chunk_count)
else:
    print("No response stream received.")

Time to first token: 0:00:00.449056
The sun had just begun to rise over the jagged peaks of the Himalayas, casting a golden hue on the rugged landscape of Ladakh. Maya had always dreamt of visiting this remote and mystical region, and today was the day her dream was finally coming true.

As she stepped out of the small, dusty jeep, the crisp, thin air filled her lungs, invigorating her spirit. The landscape was a breathtaking tapestry of rocky mountains, shimmering lakes, and verdant meadows. The air was tinged with the scent of pine and the distant sound of a river echoed through the valley.

Maya's first stop was the picturesque village of Lamayuru. The monastery, perched atop a hill, stood as a silent sentinel overlooking the village. Inside, the walls were adorned with intricate murals depicting scenes from Buddhist lore. Monks in maroon robes moved with serene grace, their chants blending harmoniously with the gentle breeze.

Next, she ventured to the stunning Pangong Tso Lake, a 

#### Image Understanding

In [28]:
import base64


with open("../images/train_snow.jpg", "rb") as f:
    snow_base64_string = base64.b64encode(f.read()).decode("utf-8")

system_list = [
    {
        "text": "You are an expert artist. When the user provides you with an image, provide three potential art titles."
    }
]

message_llist = [
    {
        "role": "user",
        "content": [
            {"image": {"format": "jpg", "source": {"bytes": snow_base64_string}}},
            {"text": "Provide art titles for this image."},
        ],
    }
]

config = {"max_new_tokens": 300, "top_k": 20, "top_p": 0.1}

request = {"messages": message_llist, "system": system_list, "inferenceConfig": config}

response = bedrock_runtime_client.invoke_model(
    modelId=LITE_MODEL_ID, body=json.dumps(request)
)
response = json.loads(response["body"].read())

content_text = response["output"]["message"]["content"][0]["text"]
print(content_text)

1. **"Winter's Embrace: The Amtrak Journey"**
2. **"Snowbound Station: A Winter's Tale"**
3. **"Amtrak's Path Through the Snow"**


In [32]:
with open("../images/desert.jpg", "rb") as f:
    desert_base64_string = base64.b64encode(f.read()).decode("utf-8")

message_llist = [
    {
        "role": "user",
        "content": [
            {"image": {"format": "jpg", "source": {"bytes": snow_base64_string}}},
            {"image": {"format": "jpg", "source": {"bytes": desert_base64_string}}},
            {"text": "What do these two images have in common?"},
        ],
    }
]

config = {"max_new_tokens": 300, "top_p": 0.1, "top_k": 20}
request = {"messages": message_llist, "inferenceConfig": config}

response = bedrock_runtime_client.invoke_model(
    modelId=LITE_MODEL_ID, body=json.dumps(request)
)

response = json.loads(response["body"].read())
content_text = response["output"]["message"]["content"][0]["text"]
print(content_text)

Both images depict landscapes, but they showcase different environments and settings. The first image features a snowy landscape with a train station and a train on the tracks, while the second image presents a desert landscape with a sunset and a unique rock formation. Despite the differences in scenery, both images share the common theme of capturing the beauty and diversity of nature.


#### Video Understanding